## PyMySQL
- MySQL을 Python에서 사용할 수 있게 하는 라이브러리

pymysql 설치

In [1]:
# ! 대신 % 사용하기 - 더 원활하게 가상환경에 인스톨됨
%pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pymysql

유저 정보가 담긴 config파일을 호출해서 dictionary 자료형에 넣는 함수 생성

In [3]:
def read_config(p:str) -> dict:
    """
    p: config file 경로
    """
    with open('./test.config', 'r') as f:
        lines = f.readlines()

    config_dict={}

    for l in lines:
        # k, v = l.split('=') # 일반적인 경우 다른 '='값이 없을 경우
        idx = l.index('=') # config파일에 다른 '=' 값이 존재할 경우 대비
        k = l[:idx]
        v = l[idx+1:]
        config_dict[k] = v.strip()

    return config_dict   

In [196]:
db_config = read_config('./test.config')

In [197]:
db_config

{'host': 'Localhost',
 'user': 'user',
 'password': 'user',
 'database': 'study',
 'charset': 'utf8mb4'}

---

### 데이터베이스 연결
- pymysql.connect() 메서드를 통해 DB에 접속가능
- host : 연결할 데이터베이스 주소
- user : 접속할 유저이름
- password : 비밀번호
- database : 접속할 DB 이름
- charset : DB 문자열 (한글- utf8, 한글 + 이모티콘- utf8mb4)
- DB 연결 해제 시, connection.close()


In [9]:
# 데이터베이스 연결
try:
    # conn = pymysql.connect(
    #     host= 'localhost',
    #     port= 3306, # default값이라 생략 가능
    #     user= 'username, # MYSQL에서 생성한 username
    #     password= 'password', # 기존에 등록한 password
    #     database= 'study',
    #     charset= 'utf8mb4'
    # )
    conn = pymysql.connect(**config_dict) # 유저 정보가 담긴 dictionary unpacking
    print('연결 성공')
except:
    print('연결 실패')

연결 성공


In [10]:
conn

---

## INSERT

In [11]:
with conn.cursor() as cursor:
    sql = 'INSERT INTO `students`(`Name`, `Age`, `Address`)\
            VALUES ("SON", "29", "서울")'
    cursor.execute(sql)

conn.commit()

In [12]:
with conn.cursor() as cursor:
    sql = 'INSERT INTO `students`(`Name`, `Age`, `Address`)\
            VALUES (%s, %s, %s)'
    cursor.execute(sql, ("LEE","30","전주"))

conn.commit()

In [13]:
with conn.cursor() as cursor:
    sql = 'INSERT INTO `students`(`Name`)\
            VALUES (%s)'
    cursor.execute(sql, ("KIM",))  #type("KIM") -> str / type("KIM", ) -> tuple

conn.commit()

## INSERT MANY

In [14]:
with conn.cursor() as cursor:
    sql = 'INSERT INTO `students`(`Name`)\
            VALUES (%s)'
    cursor.executemany(sql, [
        ("SEO"),
        ("PARK"),
    ])

conn.commit()

## SELECT

In [15]:
with conn.cursor() as cur:
    sql = 'SELECT * FROM Students'
    cur.execute(sql)
    data = cur.fetchall() # db에 commit 하지 않고 cursor에서 fetch한다

data

((1, '홍길동', 24, '인천'),
 (2, '이연걸', 60, '서울'),
 (3, '이몽룡', 42, '대전'),
 (4, '성춘향', 30, '경기'),
 (9, 'PARK', 26, None),
 (10, 'SON', 29, '서울'),
 (11, 'LEE', 30, '전주'),
 (12, 'KIM', None, None),
 (13, 'SEO', None, None),
 (14, 'PARK', None, None),
 (15, 'SON', 29, '서울'),
 (16, 'LEE', 30, '전주'),
 (17, 'KIM', None, None),
 (18, 'SEO', None, None),
 (19, 'PARK', None, None))

In [16]:
with conn.cursor() as cur:
    sql = 'SELECT * FROM Students'
    cur.execute(sql)
    data = cur.fetchmany(3) # 첫 3개 정보만 불러오기_1

data

((1, '홍길동', 24, '인천'), (2, '이연걸', 60, '서울'), (3, '이몽룡', 42, '대전'))

In [17]:
with conn.cursor() as cur:
    sql = 'SELECT * FROM Students LIMIT 3 OFFSET 0' # 첫 3개 정보만 불러오기_2
    cur.execute(sql)
    data = cur.fetchall() 

data

((1, '홍길동', 24, '인천'), (2, '이연걸', 60, '서울'), (3, '이몽룡', 42, '대전'))

In [18]:
with conn.cursor() as cur:
    sql = 'SELECT * FROM Students LIMIT %s OFFSET %s' # 첫 3개 정보만 불러오기_3
    cur.execute(sql, (3, 0))
    data = cur.fetchall() 

data

((1, '홍길동', 24, '인천'), (2, '이연걸', 60, '서울'), (3, '이몽룡', 42, '대전'))

In [19]:
page_size = 3
page = 2

limit = page_size
offset = page_size * (page-1)


with conn.cursor() as cur:
    sql = 'SELECT * FROM Students LIMIT %s OFFSET %s' # 첫 3개 정보만 불러오기_4
    cur.execute(sql, (limit, offset))
    data = cur.fetchall() 

data

((4, '성춘향', 30, '경기'), (9, 'PARK', 26, None), (10, 'SON', 29, '서울'))

SELECT - 페이지네이션 함수 생성

In [20]:
def pagination(page, page_size=5):
    limit = page_size
    offset = page_size * (page-1)


    with conn.cursor() as cur:
        sql = 'SELECT * FROM Students LIMIT %s OFFSET %s' # 첫 3개 정보만 불러오기_4
        cur.execute(sql, (limit, offset))
        data = cur.fetchall() 
    
    return data

page_one = pagination(1)
page_one


((1, '홍길동', 24, '인천'),
 (2, '이연걸', 60, '서울'),
 (3, '이몽룡', 42, '대전'),
 (4, '성춘향', 30, '경기'),
 (9, 'PARK', 26, None))

## UPDATE

In [21]:
with conn.cursor() as cur:
    sql = 'UPDATE `Students` SET Age=%s WHERE StudentID=%s'
    cur.execute(sql, (26, 9)) #(Age, studentID)

conn.commit()

In [22]:
pagination(1)

((1, '홍길동', 24, '인천'),
 (2, '이연걸', 60, '서울'),
 (3, '이몽룡', 42, '대전'),
 (4, '성춘향', 30, '경기'),
 (9, 'PARK', 26, None))

In [23]:
pagination(2)

((10, 'SON', 29, '서울'),
 (11, 'LEE', 30, '전주'),
 (12, 'KIM', None, None),
 (13, 'SEO', None, None),
 (14, 'PARK', None, None))

## UPDATE MANY

In [24]:
with conn.cursor() as cur:
    sql = 'UPDATE `Students` SET Age=%s WHERE StudentID=%s'
    cur.executemany(sql, [
        (24, 1),
        (30, 4)
    ])

conn.commit()

In [25]:
pagination(1,10)

((1, '홍길동', 24, '인천'),
 (2, '이연걸', 60, '서울'),
 (3, '이몽룡', 42, '대전'),
 (4, '성춘향', 30, '경기'),
 (9, 'PARK', 26, None),
 (10, 'SON', 29, '서울'),
 (11, 'LEE', 30, '전주'),
 (12, 'KIM', None, None),
 (13, 'SEO', None, None),
 (14, 'PARK', None, None))

## DELETE

In [26]:
with conn.cursor() as cur:
    sql = 'DELETE FROM `Students` WHERE StudentID=%s' # 예약어와 중첩 사용 방지 - 테이블명, 칼럼명에 backtick 사용
    cur.execute(sql, (6,)) # 인자가 하나이면 콤마 사용해서 튜플로 처리하기

conn.commit()

In [27]:
pagination(1,12)

((1, '홍길동', 24, '인천'),
 (2, '이연걸', 60, '서울'),
 (3, '이몽룡', 42, '대전'),
 (4, '성춘향', 30, '경기'),
 (9, 'PARK', 26, None),
 (10, 'SON', 29, '서울'),
 (11, 'LEE', 30, '전주'),
 (12, 'KIM', None, None),
 (13, 'SEO', None, None),
 (14, 'PARK', None, None),
 (15, 'SON', 29, '서울'),
 (16, 'LEE', 30, '전주'))

In [28]:
# 외래키 제약 조건 해제
# SET_FOREIGN_KEY_CHECKS = 0;
# with conn.cursor() as cur:
#     sql = 'SET FOREIGN_KEY_CHECKS = 0'
#     cur.execute(sql)

# conn.commit()

In [29]:
with conn.cursor() as cur:
    sql = 'DELETE FROM `Students` WHERE StudentID IN(%s, %s, %s)' # in 사용해서 복수의 row 삭제
    cur.execute(sql, (5, 7, 8)) # 인자가 하나이면 콤마 사용해서 튜플로 처리하기

conn.commit()

In [30]:
pagination(1,12)

((1, '홍길동', 24, '인천'),
 (2, '이연걸', 60, '서울'),
 (3, '이몽룡', 42, '대전'),
 (4, '성춘향', 30, '경기'),
 (9, 'PARK', 26, None),
 (10, 'SON', 29, '서울'),
 (11, 'LEE', 30, '전주'),
 (12, 'KIM', None, None),
 (13, 'SEO', None, None),
 (14, 'PARK', None, None),
 (15, 'SON', 29, '서울'),
 (16, 'LEE', 30, '전주'))

DELETE - 함수 생성

In [32]:
def delete(target_ids):
    with conn.cursor() as cur:
        sql = 'DELETE FROM `Students` WHERE StudentID IN (' +\
            ','.join(['%s'] * len(target_ids)) + ')'
        cur.execute(sql, target_ids)

    conn.commit()
    return

delete((10,11,))

In [34]:
delete((12,)) # 인자 하나만 삭제할 때도 콤마 사용해서 튜플로 처리

In [35]:
pagination(1,20)

((1, '홍길동', 24, '인천'),
 (2, '이연걸', 60, '서울'),
 (3, '이몽룡', 42, '대전'),
 (4, '성춘향', 30, '경기'),
 (9, 'PARK', 26, None),
 (13, 'SEO', None, None),
 (14, 'PARK', None, None),
 (15, 'SON', 29, '서울'),
 (16, 'LEE', 30, '전주'),
 (17, 'KIM', None, None),
 (18, 'SEO', None, None),
 (19, 'PARK', None, None))

---

## 실습 (PyMySQL)
- db에 student 테이블을 생성
- id (빈 값 허용 안함, 자동 증가)
- name (최대 16글자)
- email (최대 32글자)
- phone (최대 16글자)
- major (최대 16글자)
- 수강생 중 한 분의 데이터를 DB에 입력
- 수강생 중 네 분의 데이터를 DB에 동시에 추가
- 입력한 전체 데이터를 확인
- 한 수강생분의 이메일이 잘못 입력되었다고 가정하고, 이메일을 수정해서 DB에 반영
- 수강생 한 분이 취업하셨습니다. DB에서 삭제

테이블 생성

In [75]:
with conn.cursor() as cur:
    sql = """CREATE TABLE Student(
        id INT NOT NULL AUTO_INCREMENT,
        name VARCHAR(16),
        email VARCHAR(32),
        phone VARCHAR(16),
        major VARCHAR(16),
        PRIMARY KEY(id)
    )"""
    cur.execute(sql,)

conn.commit()


테이블에 데이터 추가 - INSERT 

In [76]:
with conn.cursor() as cur:
    sql = 'INSERT INTO `student`(`name`, `major`)\
        VALUES("CHOI", "Design")'
    cur.execute(sql)

conn.commit()


테이블에 복수의 데이터 추가 - INSERT MANY

In [77]:
with conn.cursor() as cur:
    sql = 'INSERT INTO `student`(`name`, `major`)\
        VALUES(%s, %s)'
    cur.executemany(sql, [      # 복수의 데이터는 리스트에 묶어서 추가하기
        ('LEE', 'English'),
        ('SEO', 'Math'),
        ('KO', 'Education'),
        ('SON', 'Sport')
    ])

conn.commit()

> 강사님 풀이 - INSERT

In [85]:
table_name = "Student"
columns = ['name', 'email', 'phone', 'major']
','.join(columns)

values = ("JOO", "j@j.com", "0101111", "Korean")

with conn.cursor() as cur:
    ['%s'] * len(values[0])
    sql = 'INSERT INTO `{}`({}) VALUES ({})'.format(table_name,
                                                    ','.join(columns),
                                                    ','.join(['%s'] * len(values)))
    cur.execute(sql, values)

conn.commit()

> 강사님 풀이 - insert 함수 생성

In [92]:
def insert(table_name:str, columns:list, values:tuple):
    """
    table_name의 columns 항목들에 values를 입력해준다.
    """
    try:
        with conn.cursor() as cur:
            ','.join(['%s'] * len(values))
            sql = 'INSERT INTO `{}`({}) VALUES ({})'.format(table_name,
                                                            ','.join(columns),
                                                            ','.join(['%s'] * len(values)))
            cur.execute(sql, values)

        conn.commit()
        return True
    except Exception as e:
        print(e)
        return False


In [93]:
table_name = 'Student'
columns = ['name', 'email', 'phone', 'major']
values = ("PARKJISUNG", "p@p.com", "0104554", "CS") # 튜플

insert(table_name, columns, values)

True

> 강사님 풀이 - INSERT MANY

In [100]:
table_name = "Student"
columns = ['name', 'email', 'phone', 'major']
values = [
    ("AHN", "", "", ""),
    ("CHAE", "", "", ""),
    ("BOO", "", "", ""),
    ("JIN", "", "", "")
]

table_name, '.'.join(columns), ','.join(['%s'] * len(values[0]))

with conn.cursor() as cur:
    sql = 'INSERT INTO `{}`({}) VALUES ({})'.format(table_name,
                                                    ','.join(columns),
                                                    ','.join(['%s'] * len(values[0])))
    cur.executemany(sql, values)

conn.commit()

> 강사님 풀이 - insertmany 함수 생성

In [191]:
def insertmany(table_name:str, columns:list, values:list):
    """
    table_name의 columns 항목들에 values를 입력해준다.
    """
    try:
        with conn.cursor() as cur:
            sql = 'INSERT INTO `{}`({}) VALUES ({})'.format(table_name,
                                                            ','.join(columns),
                                                            ','.join(['%s'] * len(values[0])))
            cur.executemany(sql, values)

        conn.commit()
        return True
    except Exception as e:
        print(e)
        return False

---

테이블의 데이터 확인 - SELECT
- commit 활용 x 
- cursor.fetch 사용

In [60]:
with conn.cursor() as cur:
    sql = 'SELECT * FROM `student`'
    cur.execute(sql)
    result = cur.fetchall()

result

((1, 'CHOI', None, None, 'Design'),
 (2, 'LEE', None, None, 'English'),
 (3, 'SEO', None, None, 'Math'),
 (4, 'KO', None, None, 'Education'),
 (5, 'SON', None, None, 'Sport'))

 > 강사님 풀이 - SELECT

In [106]:
# columns = ['id', 'name']   # 특정 컬럼의 값만 가져올 때
columns = ['*']              # 컬럼 모두 가져올 때
table_name = 'Student'

with conn.cursor() as cur:
    sql = 'SELECT {} FROM {}'.format(','.join(columns), table_name)
    cur.execute(sql)
    data = cur.fetchall()

In [105]:
data

((1, 'CHOI', None, None, 'Design'),
 (2, 'LEE', None, None, 'English'),
 (3, 'SEO', None, None, 'Math'),
 (4, 'KO', None, None, 'Education'),
 (5, 'SON', None, None, 'Sport'),
 (6, 'JOO', 'j@j.com', '0101111', 'Korean'),
 (7, 'PARKJISUNG', 'p@p.com', '0104554', 'CS'),
 (8, 'AHN', '', '', ''),
 (9, 'CHAE', '', '', ''),
 (10, 'BOO', '', '', ''),
 (11, 'JIN', '', '', ''))

---

테이블에 데이터 추가 - UPDATE

In [61]:
with conn.cursor() as cur:
    sql = 'UPDATE `student` SET `email`=%s WHERE `id`=%s'
    cur.execute(sql, ('sesac@python.org', 5))

conn.commit()

> 강사님 풀이 - UPDATE

In [118]:
tabale_name = 'Student'
set_value = 'ko@py.org'
target_id = 4

with conn.cursor() as cur:
    sql = 'UPDATE `{}` SET `email`="{}" WHERE `id`={}'.format(
        table_name, set_value, target_id
    )
    cur.execute(sql)
conn.commit()

테이블에 저장된 데이터 삭제 - DELETE FROM

In [63]:
with conn.cursor() as cur:
    sql = 'DELETE FROM `student` WHERE `id`=%s'
    cur.execute(sql, (4,))

conn.commit()

In [64]:
pagination1(1)

((1, 'CHOI', None, None, 'Design'),
 (2, 'LEE', None, None, 'English'),
 (3, 'SEO', None, None, 'Math'),
 (5, 'SON', 'sesac@python.org', None, 'Sport'))

---

## CLASS

In [239]:
class MariaDB:
    """
    MariaDB를 관리하는 클래스
    """

    def __init__(self, host, user, password, database, charset, port=3306):
        self.host = host
        self.port = int(port) # 포트명은 int
        "다른 함수에서 사용하기 위해서 self에 담아준다"
        self.conn = pymysql.connect(
                                host=host,
                                user=user,
                                password=password,
                                database=database,
                                charset=charset,
                                port=port)  # 포트명은 int
        return
    
    
    def insert(self, table_name:str, columns:list, values:tuple):
        try:    
            with self.conn.cursor() as cur:
                ','.join(['%s'] * len(values))
                sql = 'INSERT INTO {} ({}) VALUES ({})'.format(table_name,
                                                                ','.join(columns),
                                                                ','.join(['%s'] * len(values)))
                cur.execute(sql, values) 

            self.conn.commit()
            return True
        except Exception as e:
            print(e)
            return False


    def insertmany(self, table_name:str, columns:list, values:list):
        try:
            with self.conn.cursor() as cur:
                sql = 'INSERT INTO {} ({}) VALUES ({})'.format(table_name,
                                                                ','.join(columns),
                                                                ','.join(['%s'] * len(values[0])))
                cur.executemany(sql, values)

            self.conn.commit()
            return True
        except Exception as e:
            print(e)
            return False



    def select(self, table_name, columns, page=None, page_size=10): # pagination 기능 추가
        if page:
            limit = page_size
            offset = page_size * (page-1)
            with self.conn.cursor() as cur:
                sql = 'SELECT {} FROM {} LIMIT %s OFFSET %s'.format(','.join(columns), table_name) 
                cur.execute(sql, (limit, offset))
                data = cur.fetchall() 

        else: # page가 None일 경우 False로 인식
            with self.conn.cursor() as cur:
                sql = 'SELECT {} FROM {}'.format(','.join(columns), table_name)
                cur.execute(sql)
                data = cur.fetchall()

        return data



    def update(self, table_name, set_column, set_value, target_id):
        try:
            with self.conn.cursor() as cur:
                sql = 'UPDATE {} SET {}=%s WHERE id={}'.format(
                    table_name, set_column, target_id
                )
                cur.execute(sql, (set_value,))
            self.conn.commit()
            return True
        except Exception as e:
            print(e)
            return False

In [244]:
db= MariaDB(**db_config) # test
# db1 = MariaDB(**db_config) # production

In [245]:
db.conn

insert 함수 동작 확인

In [204]:
db.insert('student', ['name', 'major'], ('JANG', 'Chemistry'))

True

insertmany 함수 동작 확인

In [248]:
table_name = "Student"
columns = ['name', 'email', 'phone', 'major']
values = [
    ("KANG", "e", "e", "e"),
    ("JUNG", "e", "e", "e")
]
db.insertmany(table_name, columns, values)

True

select 함수 동작 확인

In [226]:
db.select(table_name, columns, page=5, page_size=3)

(('JANG', None, None, 'Chemistry'),
 ('JANG', None, None, 'Chemistry'),
 ('KANG', 'e', 'e', 'e'))

update 함수 동작 확인

In [247]:
table_name = "Student"
set_column = 'phone'
set_value = '1212'
target_id = '43'

db.update(table_name, set_column, set_value, target_id)

True